In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import random
from utlis import tools
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset,DataLoader
from torch import nn, optim

import os

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore')

import time

In [ ]:
# pip install tsaug
from tsaug.visualization import plot
from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse, AddNoise
from multiprocessing import Pool

## 1. Load Preprocessed Data

In [ ]:
# path to the data
dataPath = './dataset/data_combined.csv'

# size of sliding window
lagWindowSize=20

In [ ]:
def load_n_preprocess_data(dataPath, onlyHand=False, lagWindowSize=10):
    ## 1. Load labelled data
    print("Load Labelled Data")
    X, y = tools.load_labelled_data(dataPath, onlyHand)
    ## 2. Generate Time-lagged data
    print("Generate Time-lagged Data")
    X_lag, y_lag = tools.generate_time_lags(X, y, lagWindowSize)
    ## 3. Convert 2D dataframe to 3D numpy array: (Batch, TimeLag, Features)
    print("Convert 2D dataframe to 3D numpy array")
    X_3D = tools.convert_df_2_np_3D(X_lag, lagWindowSize)
    return X_3D, y_lag

In [ ]:
# Save/Load data
if(os.path.exists('./dataset/X_{}.npy'.format(lagWindowSize))):
    X_3D = np.load('./dataset/X_{}.npy'.format(lagWindowSize))
    y_lag = np.load('./dataset/y_{}.npy'.format(lagWindowSize))
else:
    start = time.time()
    print("Load and Preprocessing")
    X_3D, y_lag = load_n_preprocess_data(dataPath,
                                         onlyHand=False,
                                         lagWindowSize=lagWindowSize)
    y_lag = y_lag.to_numpy()
    np.save('./dataset/X_{}.npy'.format(lagWindowSize), X_3D)
    np.save('./dataset/y_{}.npy'.format(lagWindowSize), y_lag)

    end = time.time()
    elapsed_time = end - start
    print('Execution time:', elapsed_time, 'seconds')

In [ ]:
# if we only consider one-hand guesture
def ignore_left_n_right(y):
    # for catheter holding
    y[np.where(y==4)] = 1
    # for catheter insertion
    y[np.where(y==5)] = 2
    # for stylet removal
    y[np.where(y==6)] = 3
    return y

In [ ]:
output_dimension = 7
classify_both_hand = True
if classify_both_hand:
    y_lag = ignore_left_n_right(y_lag)
    output_dimension = 4

In [ ]:
X_3D.shape

## 2. Split data into training & test set

In [ ]:
def split_train_n_test(X, y, test_size=0.4, val_size=0.1):
    # Convert dataframe to numpy
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size,
                                                        shuffle=False,
                                                        random_state=2022)
    val_size = val_size/(1-test_size)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                      test_size=val_size, 
                                                      shuffle=False,
                                                      random_state=2022)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = split_train_n_test(
                                                      X_3D, y_lag, 
                                                      test_size=0.1,
                                                      val_size=0.1
                                                     )

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

## 3. Convert datasets into pyTorch format 

In [ ]:
ts_augmenter = (
    #TimeWarp(n_speed_change=2, max_speed_ratio=2) * 1  # random time warping 5 times in parallel
    #+ Quantize(n_levels=[10, 20, 30])  # random quantize to 10-, 20-, or 30- level sets
    #+ Drift(max_drift=(0, 0.1)) @ 0.5  # with 80% probability, random drift the signal up to 10% - 50%
    AddNoise(scale=0.05)
)

def augment_feature(x):
    return ts_augmenter.augment(x)

In [ ]:
from utlis import augmentation as aug
import random

In [ ]:
def augment_feature_with_prob(X, prob):
    if(random.random()<prob):
        X = X.reshape((1, X.shape[0], X.shape[1]))
        X = aug.window_warp(X, 0.2, 3)
        X = aug.jitter(X, 0.1)
        X = np.squeeze(X)
    return X

In [ ]:
class torchDataset(Dataset):
    def __init__(self, X, y, augment_flag=False, num_processes=16):
        self.X = X
        self.y = y
        self.augment_flag = augment_flag
        self.num_processes = num_processes

    @staticmethod
    def augment(X, num_processes):
        #X_aug = ts_augmenter.augment(X)
        X_aug = augment_feature_with_prob(X, 0.2)
        #X_aug = np.stack([ts_augmenter.augment(x) for x in X])
#         X_2_aug = [x for x in X]
#         with Pool(num_processes) as pool:
#             X_aug = np.stack(pool.map(augment_feature, X_2_aug))
        return X_aug
    
    @staticmethod
    def to_torch_tensor(X, y):
        X = torch.tensor(X).float()
        y = torch.tensor(y).float()
        return X,y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i):
        Xi, yi = self.X[i], self.y[i]
        if(self.augment_flag):
            Xi = torchDataset.augment(Xi, self.num_processes)
        Xi,yi = torchDataset.to_torch_tensor(Xi,yi)
        return Xi,yi

In [ ]:
trainSet = torchDataset(X_train, y_train, True, 1)
valSet = torchDataset(X_val, y_val, True)
testSet = torchDataset(X_test, y_test)

In [ ]:
np.max(y_train)

In [ ]:
from torch.utils.data import DataLoader
torch.manual_seed(42)

batch_size = 64

train_loader = DataLoader(trainSet, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valSet, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testSet, batch_size=1, shuffle=False)

In [ ]:
%%time
X, Y = next(iter(train_loader))
print("Batch Features shape:", X.shape)
print("Batch Target shape:", Y.shape)

In [ ]:
X.shape

## 4. Import models

In [ ]:
# check GPU availability               
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Run on {}...".format(device))

In [ ]:
from models import GRU, CNN2DGRU

In [ ]:
'''
model = GRU.GRUModel(input_dim=X.shape[-1], 
                    hidden_dim=10, 
                    layer_dim=3, 
                    output_dim=2, 
                    dropout_prob=0.5)
'''

In [ ]:
X.shape[-1]

In [ ]:
model = CNN2DGRU.CNN2DGRUModel(input_dim=X.shape[-1], 
                    hidden_dim=20, 
                    layer_dim=3, 
                    output_dim=output_dimension, 
                    dropout_prob=0.5)

## 5. Train, Update, and Test

In [ ]:
def train(dataloader, model, criterion, optimizer, scheduler, device):
    model.train()
    epoch_losses = []
    #epoch_mses = []
    epoch_accs = []
    
    for X, y in dataloader:
        X, y = X.to(device), y.type(torch.LongTensor).to(device)
        #X, y = X.to(device), y.type(torch.float32).to(device)
        #X[:, :, 93:168] = 0
        y_hat = model(X)
        loss = criterion(y_hat, y.squeeze(dim=1))
        #loss = criterion(y_hat.argmax(dim=-1), y.squeeze(dim=1))
        accuracy = get_accuracy(y_hat, y.squeeze(dim=1))
        #real_mse = real_mean_square_error(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        epoch_losses.append(loss.item())
        #epoch_mses.append(real_mse)
        epoch_accs.append(accuracy.item())
        scheduler.step()
    return epoch_losses, epoch_accs#epoch_mses

def evaluate(dataloader, model, criterion, device):
    model.eval()
    epoch_losses = []
    #epoch_mses = []
    epoch_accs = []
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.type(torch.LongTensor).to(device)
            #X, y = X.to(device), y.type(torch.float32).to(device)
            #X[:, :, 93:168] = 0
            y_hat = model(X)
            loss = criterion(y_hat, y.squeeze(dim=1))
            #loss = criterion(y_hat.argmax(dim=-1), y.squeeze(dim=1))
            accuracy = get_accuracy(y_hat, y.squeeze(dim=1))
            #real_mse = real_mean_square_error(y_hat, y)
            epoch_losses.append(loss.item())
            #epoch_mses.append(real_mse)
            epoch_accs.append(accuracy.item())
    return epoch_losses, epoch_accs#epoch_mses

def get_accuracy(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy

In [ ]:
class ConstantWithWarmup(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
        self,
        optimizer,
        num_warmup_steps: int,
    ):
        self.num_warmup_steps = num_warmup_steps
        super().__init__(optimizer)

    def get_lr(self):
        if self._step_count <= self.num_warmup_steps:
            # warmup
            scale = 1.0 - (self.num_warmup_steps - self._step_count) / self.num_warmup_steps
            lr = [base_lr * scale for base_lr in self.base_lrs]
            self.last_lr = lr
        else:
            # every 10 steps, exponentially decay by multipling 0.95
            if self._step_count % 2000 == 0:
                self.base_lrs = [base_lr * 0.9 for base_lr in self.base_lrs]
                print("Learning Rate Decay - lr: {}".format(self.base_lrs[0]))
            lr = self.base_lrs
        return lr

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def loss_weight_balance(DataLoader):
    label_range = output_dimension
    Y = torch.Tensor([])
    for _, y in DataLoader:
        Y = torch.cat((Y, y.squeeze(dim=1)))
    Y = Y.numpy()
    totalNum = len(Y)
    ratios = np.array([])
    for i in range(label_range):
        ratio = np.sum((Y == float(i))*1)/totalNum        
        ratios = np.append(ratios, ratio)
    print("Label Distribution: {}".format(ratios))
    ratios = ratios
    weights = np.array([])
    for i in range(label_range):       
        weights = np.append(weights, 1.0/ratios[i])
    weights = weights/np.sum(weights)
    print("Label Weights: {}".format(weights))
    return weights

In [ ]:
trainSet.__getitem__(i=0)[0].shape

In [ ]:
def train_and_test_model_with_hparams(hparams, model_type="lstm", **kwargs):

    torch.manual_seed(hparams.SEED)
    random.seed(hparams.SEED)
    np.random.seed(hparams.SEED)
    
    train_loader = DataLoader(trainSet, batch_size=hparams.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(valSet, batch_size=hparams.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(testSet, batch_size=hparams.BATCH_SIZE, shuffle=True)
    
    X, _ = trainSet.__getitem__(i=0)
    feature_size = X.shape[1]
    
    # Model
    if model_type=="gru":
        model = GRU.GRUModel(
            input_dim = feature_size, 
            hidden_dim = hparams.HIDDEN_DIM, 
            output_dim = hparams.OUTPUT_DIM,
            layer_dim = hparams.N_LAYERS,
            dropout_prob = hparams.DROPOUT_RATE, 
            **kwargs)
    elif model_type=="cnn2dgru":
        model = CNN2DGRU.CNN2DGRUModel(
            input_dim = feature_size, 
            hidden_dim = hparams.HIDDEN_DIM, 
            output_dim = hparams.OUTPUT_DIM,
            layer_dim = hparams.N_LAYERS,
            dropout_prob = hparams.DROPOUT_RATE, 
            **kwargs)
        
    num_params = count_parameters(model)
    print(f'The model has {num_params:,} trainable parameters')

    model = model.to(device)

    if hparams.OPTIM == "sgd":
        optimizer = optim.SGD(
            model.parameters(), lr=hparams.LR, weight_decay=hparams.WD, momentum=.9)        
    elif hparams.OPTIM == "adagrad":
        optimizer = optim.Adagrad(
            model.parameters(), lr=hparams.LR, weight_decay=hparams.WD, eps=1e-6)
    elif hparams.OPTIM == "adam":
        optimizer = optim.Adam(
            model.parameters(), lr=hparams.LR, weight_decay=hparams.WD, eps=1e-6)
    elif hparams.OPTIM == "rmsprop":
        optimizer = optim.RMSprop(
            model.parameters(), lr=hparams.LR, weight_decay=hparams.WD, eps=1e-6, momentum=.9)
    else:
        raise NotImplementedError("Optimizer not implemented!")

    
    ## ! Balance the loss for imbalanced dataset
    weights = loss_weight_balance(train_loader)
    #weights[1] = weights[1]*10
    #weights[3] = weights[3]*5
    weights = torch.Tensor(weights)
    criterion = nn.CrossEntropyLoss(weight=weights)
    #criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(device)

    # Start training
    best_valid_loss = float('inf')
    train_losses = []
    train_accs = []
    valid_losses = []
    valid_accs = []
    
    # Warmup Scheduler.
    WARMUP_STEPS = 100
    lr_scheduler = ConstantWithWarmup(optimizer, WARMUP_STEPS)

    train_loss_records = []
    valid_loss_records = []
    
    for epoch in range(hparams.N_EPOCHS):
        
        # Your code: implement the training process and save the best model.
        train_losses, train_acc = train(dataloader=train_loader, 
                                      model=model, 
                                      criterion=criterion, 
                                      optimizer=optimizer, 
                                      scheduler=lr_scheduler, 
                                      device=device)
        valid_losses, valid_acc = evaluate(dataloader=val_loader,
                                         model=model,
                                         criterion=criterion,
                                         device=device)
        
        #print(train_real_mses.shape)
        epoch_train_loss = np.mean(train_losses)
        epoch_train_acc = np.mean(train_acc)
        epoch_valid_loss = np.mean(valid_losses)
        epoch_valid_acc = np.mean(valid_acc)
        train_loss_records += train_losses
        valid_loss_records += valid_losses
        

        # Save the model that achieves the smallest validation loss.
        if epoch >=5:
            if epoch_valid_loss < best_valid_loss:
                # Your code: save the best model somewhere (no need to submit it to Sakai)
                best_valid_loss = epoch_valid_loss
                if not os.path.exists(hparams.CHECKPOINT_FOLDER):
                    os.makedirs(hparams.CHECKPOINT_FOLDER)
                print("Saving ...")
                state = {'state_dict': model.state_dict(),
                         'epoch': epoch}
                torch.save(state, os.path.join(hparams.CHECKPOINT_FOLDER, model_type+'.pth'))
        if epoch%5 == 0:
            print(f'epoch: {epoch+1}')
            print(f'train_loss: {epoch_train_loss:.5f}, train_acc: {epoch_train_acc:.5f}')
            print(f'valid_loss: {epoch_valid_loss:.5f}, valid_acc: {epoch_valid_acc:.5f}')


    # Your Code: Load the best model's weights.
    state_dict = torch.load('./saved_model/'+model_type+'.pth')['state_dict']
    model.load_state_dict(state_dict)
    model.to(device)
    
    # Your Code: evaluate test loss on testing dataset (NOT Validation)
    test_losses, test_acc = evaluate(dataloader=test_loader, 
                                   model=model,
                                   criterion=criterion,
                                   device=device)

    epoch_test_loss = np.mean(test_losses)
    epoch_test_acc = np.mean(test_acc)
    print(f'test_loss: {epoch_test_loss:.5f}, test_acc: {epoch_test_acc:.5f}')
    
    # Free memory for later usage.
    #del model
    #torch.cuda.empty_cache()
    return {
        'model': model,
        'num_params': num_params,
        "test_loss": epoch_test_loss,
        "test_acc": epoch_test_acc,
        
        'train_loss_records': train_loss_records,
        'valid_loss_records': valid_loss_records
    }

In [ ]:
class HyperParams:
    def __init__(self):
        self.BATCH_SIZE = 64
        self.HIDDEN_DIM = 128
        self.OUTPUT_DIM = output_dimension
        self.N_LAYERS = 1
        self.DROPOUT_RATE = 0.0
        self.LR = 1e-4
        self.N_EPOCHS = 30
        self.OPTIM = "rmsprop"
        self.SEED = 2
        self.WD = 0
        self.CHECKPOINT_FOLDER = 'saved_model'
        
hparams = HyperParams()
torch.manual_seed(hparams.SEED)
random.seed(hparams.SEED)
np.random.seed(hparams.SEED)

In [ ]:
model_type = "cnn2dgru"
title = "gru_l3_h10_rmsprop"
gru_result = train_and_test_model_with_hparams(hparams, model_type)
model = gru_result['model']
#print(model)

# Evaluation

In [ ]:
def plot_train_valid_loss(result, title):
    train_loss = result['train_loss_records']
    valid_loss = result['valid_loss_records']
    valid_iter = [int((len(train_loss)/len(valid_loss))*x) \
                  for x in range(len(valid_loss))]
    plt.figure(figsize=(14, 6))
    plt.plot(train_loss, 'g', label='Training loss')
    plt.plot(valid_iter, valid_loss, 'b', label='Validation loss')
    plt.title(title)
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
plot_train_valid_loss(gru_result, title)

In [ ]:
def predict(data_loader, model):
    y_heads = torch.tensor([]).to(device)
    y_truth = torch.tensor([]) # Let it stays on CPU
    model.eval()
    #model.train()
    with torch.no_grad():
        for X, y in data_loader:
            X = X.to(device)
            y_star = model(X)
            y_heads = torch.cat((y_heads, y_star), 0)
            y_truth = torch.cat((y_truth, y), 0)
    del model
    torch.cuda.empty_cache()
    predicted_classes = y_heads.argmax(dim=-1)
    return predicted_classes.cpu().numpy(), y_truth.cpu().numpy()

def get_predict_accuracy(prediction, label):
    label = np.reshape(label, prediction.shape)
    correct_predictions = np.sum((prediction==label)*1)
    accuracy = correct_predictions / len(label)
    return accuracy

In [ ]:
train_loader = DataLoader(trainSet, batch_size=batch_size, shuffle=False)

y_train_head, y_train  = predict(train_loader, model)

start = time.time()
y_test_head, y_test = predict(test_loader, model)
end = time.time()
elapsed_time = end - start
print('Test Set Execution time:', elapsed_time, 'seconds')

In [ ]:
def visual_output(y, y_head, title):
    fig = plt.figure(figsize=(14,3))
    plt.plot(y, color='g', label='truth')
    if y_head is not None:
        plt.plot(y_head, color='b', label='predict')
        accuracy = get_predict_accuracy(y_head, y)
        plt.title(title+"- acc: {:.5f}".format(accuracy))
    #plt.ylim(-0.01, (np.mean(y_head)+np.mean(y))*2)
    else:
        plt.title(title)
    plt.xlabel('Step')
    plt.ylabel('Error')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
from sklearn import metrics
import seaborn as sns

In [ ]:
print(metrics.classification_report(y_test, y_test_head, digits=4))

In [ ]:
cm = metrics.confusion_matrix(y_test, y_test_head, normalize='true')
cm

In [ ]:
classes = ["Miscellaneous", "Holding", "Insertion", "Removal"]
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues, values_format='.3f')
plt.xticks([0,1,2,3], classes, fontsize=18, rotation=30)
plt.yticks([0,1,2,3], classes, fontsize=18, rotation=30)
for labels in disp.text_.ravel():
    labels.set_fontsize(18)
plt.show()